# Notebook activity 7.3 A GLM for predicting Olympic medals

In Unit 5, we discussed the problem of using data from the Olympic dataset to model the response variable `medals`,  the number of medals won by a nation at a summer Olympics (as it stood at the end of Tokyo 2020). In that unit, even though the response `medals` is a count variable, we used a linear model, because at that stage we only had linear models in our statistical modelling tool box!

In this notebook activity, we shall instead model `medals` using a Poisson GLM with a log link. 

The Olympic dataset includes data for the competing nations for each summer Olympics between 1996 and 2016. In Notebook activity 5.11, we used these data to fit the **linear** model 

`medals ~ lagMedals + population + log(population) + log(gdp) + host + nextHost`.

We then assessed how good these predictions were using data from Tokyo 2020 (which hadn't been used for fitting the model).

In this notebook activity, we will follow the same process using the same data and the same response and explanatory variables, but this time we'll fit a **Poisson GLM with a log link**.

As a reminder, the explanatory variables are defined as:

* `lagMedals`: the number of medals won by a nation at the previous summer Olympics (as it stood at the end of Tokyo 2020)
* `host`: whether a nation is the current host, taking the possible coded values 0 (for not current host) and 1 (for current host)
* `nextHost`: whether a nation is going to be the next host, taking the possible coded values 0 (for not next host) and 1 (for next host)
* `population`: total population size (in millions) of a nation in the year of the previous Olympics
* `gdp`: a nation's GDP per capita (in thousands), relative to the US dollar in 2010, in the year of the previous Olympics.

The training data for fitting the model are stored in the `olympic` data frame, while the test data are stored in a data frame called `olympic2020`. 

## (a) Getting started

We'll start by loading the data frames `olympic` and `olympic2020`.

### (i)

Run the code cell below to load the data frames `olympic2020` and `olympic`. The `head()` function is used to check that the two data frames both have the same format.

In [ ]:
# Load the M348 package
library(M348)
# Load the olympic and olympic 2020 data frames
data(olympic)
data(olympic2020)
# Check the data frames have loaded OK
head(olympic2020)
head(olympic)

## (b) Creating new explanatory variables

We are going to fit the model:

`medals ~ lagMedals + population + log(population) + log(gdp) + host + nextHost`

using a Poisson GLM with a log link. 

First we need to create two new explanatory variables for the training data:

`logPopulation` = `log(population)` and `logGdp` = `log(gdp)`.

### (i)

Use the empty code cell below to create the variables `logPopulation` and `logGdp` and add them to the `olympic` data frame.

#### Solution <a class="tocSkip">

The required code is given below. Running the code won't produce any output.

In [2]:
olympic$logPopulation <- log(olympic$population)
olympic$logGdp <- log(olympic$gdp)

## (c) Fitting a Poisson GLM with a log link

We are now ready to fit a Poisson GLM with a log link using the training data (stored in the `olympic` data frame).

### (i)

Use the empty code cell below to fit the model

`medals ~ lagMedals + population + logPopulation + logGdp + host + nextHost`,

using a Poisson GLM with a log link. 

Store the results from fitting the model in the object `medalsPoissonFit`.

#### Solution <a class="tocSkip">

The required code is given below. Running the code cell won't produce any output.

In [3]:
medalsPoissonFit <- glm(medals ~ lagMedals + population + logPopulation + 
                        logGdp + host + nextHost, family = poisson, 
                        data = olympic)

###  <a class="tocSkip">

Note that because we are assuming a different error distribution for `medals` when fitting the model in (i) compared with that when fitting the model in Notebook activity 5.11, it is problematical to use either adjusted $R^2$ or AIC to compare the models. However, it does make sense to compare predictions produced by these models. 

## (d) Creating and then adding new variables to the test dataset

We wish to assess the predictions obtained from our fitted model using the test data stored in the data frame `olympic2020`.

Two of the explanatory variables in the fitted model are transformations of the original data - namely, `logPopulation` and `logGdp`. We created these variables for the training data stored in the data frame `olympic` in order to fit our model. We now need to create these variables for the test data stored in the data frame `olympic2020`.

### (i)

Run the code cell below to:
* Create and add the transformed variables `logPopulation` and `logGdp` for the data in the `olympics2020` data frame to that data frame. 
* Use the `head()` function to check that the two new variables have been added to the data frame.

In [ ]:
# Creating the transformed variables for the data in olympic2020
olympic2020$logPopulation <- log(olympic2020$population)
olympic2020$logGdp <- log(olympic2020$gdp)

# Check that new variables have been added to the data frame
head(olympic2020)

#### Solution <a class="tocSkip">

Looking at resulting table, the two new variables have indeed been added to the data frame.

## (e) Assessing predictions for the fitted model

### (i)

Use the empty code cell below to obtain the predicted mean responses from the fitted model (stored in `modelFit`) for the data contained in the data frame `olympic2020`.

Store the predictions in an object called `predictions2020`.

(You might find it useful to look back at Notebook activities 5.11 and 7.2.)

#### Solution <a class="tocSkip">

The required code for calculating the predicted mean responses is given in the code cell below. Remember that for a GLM, if we want predictions of the mean responses, we need to specify that the type of predictions that we're making is for the response.

Running the code won't produce any output.

In [5]:
predictions2020 <- predict(medalsPoissonFit, newdata = olympic2020, 
                           type = "response")

### (ii)

Run the code below to calulate the MSE for the predicted mean responses that you've just calculated. What is the MSE?

In [ ]:
obs <- olympic2020[, "medals"]
mean((predictions2020 - obs)^2)

#### Solution <a class="tocSkip">

The MSE is 142.19 (rounded to two decimal places). 

### (iii)

In Notebook activity 5.11, the MSE for the predictions using these same data but using a fitted linear model instead, was calculated to be 11.92. Which of the Poisson GLM with a log link and the linear model performed better in terms of MSE for these data?

#### Solution <a class="tocSkip">

The MSE for the GLM is much larger than the MSE for the linear model. As such, the linear model performs better in terms of MSE than the GLM does.

## (f) A closer look at the predicted mean responses

It turns out that the MSE for the GLM is so much larger than the MSE for the linear model. This is due to just *one* extremely large difference between the predicted mean response and the associated observation (the observation for the United states in `olympic2020`). 

If we remove this data value and recalculate the MSE for the GLM, the value of the MSE reduces to 26.98. This is much more in line with the MSE calculated for the linear model, although it is still larger than the MSE for the linear model.

So, even though `medals` is a count response variable, and is therefore suitable for a Poisson GLM, the linear model outperforms the GLM in terms of the MSE. This illustrates that models which in theory should work better, may not do  so in practice! 

It is, however, worth noting that some of the individual predicted values of `medals` calculated using the linear model, do have negative values.